**구글 드라이브 연동**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**모델 정의, 학습 및 테스트, 저장**

In [ ]:
# 1. 라이브러리 임포트
import tensorflow
physical_devices = tensorflow.config.list_physical_devices('GPU')

try:
    tensorflow.config.experimental.set_memory_growth(physical_devices[0], True)
    tensorflow.config.experimental.set_memory_growth(physical_devices[1], True)
    #set_memory_growth를 활성화 시 TensorFlow는 필요한 만큼의 GPU 메모리만 할당하게 된다.
    #만약 이것이 False로 세팅될 경우 TensorFlow는 시작 시 GPU의 모든 메모리를 할당 하려고 할 것이며, 다른 프로세스가 해당 메모리를 사용할 수 없게 된다.
except:
    # GPU장치가 유효하지 않거나 한번 초기화된 이후 virtual devices의 세팅값을 수정할 수 없는 상황.
    print("Invalid device or cannot modify virtual devices once initialized.")
    pass

import glob
from PIL import Image, ImageDraw
import numpy as np
import json
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
import pandas as pd

print("# 1. 라이브러리 임포트")

# 2. 경로 설정
#객체 정보 xlsx의 경로
object_classification_path = r'/content/drive/MyDrive/2dss/2DSS/2D_Semantic_Segmentation/object_seg.xlsx'

#이미지 파일 경로
train_image_paths = sorted(glob.glob(r'/content/drive/MyDrive/2dss/2DSS/2D_Semantic_Segmentation/training/images/*.jpg'))
val_image_paths = sorted(glob.glob(r'/content/drive/MyDrive/2dss/2DSS/2D_Semantic_Segmentation/validation/images/*.jpg'))
test_image_paths = sorted(glob.glob(r'/content/drive/MyDrive/2dss/2DSS/2D_Semantic_Segmentation/test/image/*.jpg'))

#json 파일 경로
train_json_paths = sorted(glob.glob('/content/drive/MyDrive/2dss/2DSS/2D_Semantic_Segmentation/training/labels/*.json'))
val_json_paths = sorted(glob.glob('/content/drive/MyDrive/2dss/2DSS/2D_Semantic_Segmentation/validation/labels/*.json'))
test_json_paths = sorted(glob.glob('/content/drive/MyDrive/2dss/2DSS/2D_Semantic_Segmentation/test/labels/*.json'))
print("이미지랑 json 파일 성공적으로 가져옴")

# 3. 객체 분류 파일 읽기
object_classes_df = pd.read_excel(object_classification_path)
total_classes = len(object_classes_df)
print(total_classes)
print(object_classes_df)
print("# 2. 객체 분류 파일 읽기")

# 4. 레이블 이름에서 클래스 ID 찾기
def get_class_id(label_name, object_classes_df):
    class_id = int(object_classes_df[object_classes_df['class'].str.strip() == label_name].index[0])
    return class_id

# 5. 레이블 이미지 생성 함수
def create_label_image(annotation_data, width, height, object_classes_df):
    label_image = Image.new('L', (width, height), 0)
    draw = ImageDraw.Draw(label_image)
    for annotation in annotation_data:
        coordinates = annotation['Coordinate'][0]
        polygon = [(coordinates[i], coordinates[i + 1]) for i in range(0, len(coordinates), 2)]
        label_name = annotation['Label']
        class_id = get_class_id(label_name, object_classes_df)
        draw.polygon(polygon, outline=class_id, fill=class_id)
    label_array = np.array(label_image)
    one_hot_labels = np.eye(total_classes)[label_array]
    return one_hot_labels

# 6. 이미지와 레이블 로딩 함수
def load_images_and_labels(image_paths, json_paths, object_classes_df):
    images = []
    labels = []
    for image_path, json_path in zip(image_paths, json_paths):
        image = Image.open(image_path)
        images.append(np.array(image) / 255.0)
        with open(json_path, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
        label_image = create_label_image(json_data['Annotation'], image.width, image.height, object_classes_df)
        labels.append(label_image)
    return np.array(images), np.array(labels)

# 7. U-Net 모델 구축 함수
def build_unet(input_shape, num_classes):
    inputs = Input(input_shape)
    # Encoder
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    # Bottleneck
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    # Decoder
    up4 = concatenate([UpSampling2D(size=(2, 2))(conv3), conv2], axis=-1)
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(up4)
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv4)
    up5 = concatenate([UpSampling2D(size=(2, 2))(conv4), conv1], axis=-1)
    conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(up5)
    conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv5)
    # Output Layer
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(conv5)
    return Model(inputs=[inputs], outputs=[outputs])

def data_generator(image_paths, json_paths, object_classes_df, batch_size=32):
    while True:
        for i in range(0, len(image_paths), batch_size):
            batch_image_paths = image_paths[i:i+batch_size]
            batch_json_paths = json_paths[i:i+batch_size]
            images, labels = load_images_and_labels(batch_image_paths, batch_json_paths, object_classes_df)
            yield images, labels

# 8. 학습, 검증, 테스트 데이터 로딩
batch_size = 2

# 제너레이터 생성
train_generator = data_generator(train_image_paths, train_json_paths, object_classes_df, batch_size)
val_generator = data_generator(val_image_paths, val_json_paths, object_classes_df, batch_size)

# 9. U-Net 모델 구축
# 임시로 첫 번째 이미지를 열어서 입력 모양 확인
temp_image = Image.open(train_image_paths[0])
input_shape = (temp_image.size[1], temp_image.size[0], 3)
unet_model = build_unet(input_shape, total_classes)
print("U-NET 모델 구축 완료")

# 10. 모델 학습
unet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
unet_model.fit(train_generator,
               steps_per_epoch=len(train_image_paths) // batch_size,
               validation_data=val_generator,
               validation_steps=len(val_image_paths) // batch_size,
               epochs=1)
print("모델 학습 완료")



In [ ]:
import matplotlib.pyplot as plt

# 11. 테스트 데이터 로드
test_images, test_labels = load_images_and_labels(test_image_paths, test_json_paths, object_classes_df)

# 12. 모델 평가
evaluation = unet_model.evaluate(test_images, test_labels, batch_size=batch_size)
print("테스트 손실:", evaluation[0])
print("테스트 정확도:", evaluation[1])

# 13. 라벨링된 이미지 출력
predictions = unet_model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=-1)

plt.imshow(predicted_labels[0])
plt.show()

# 14. 모델 파일 저장
save_path = '/content/drive/MyDrive/2dss/2DSS/2D_Semantic_Segmentation/final_model.h5'
unet_model.save(save_path)
print(f"모델을 {save_path}에 저장 성공")